In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from pathlib import Path
import pandas as pd
import re
import unicodedata

# ================== PARAMS ================== #
ROOT_DIR = Path("fic")                # Dossier d'entrée (récursif)
EXPORT_DIR = Path("out")              # Dossier de sortie CSV (créé si export activé)
EXPORT_TABLES = True                  # Mettre True pour exporter les tables détectées
SHOW_SAMPLES = True                   # Afficher un aperçu (head) des tables détectées

# Heuristiques génériques (adapter si besoin)
MIN_COLS = 2               # nb min de colonnes non vides pour considérer une ligne "tabulaire"
MIN_CONSEC_ROWS = 5        # nb min de lignes consécutives "tabulaires" pour former un bloc
ROW_EMPTY_TOL = 1          # nb max de lignes vides autorisées à l'intérieur d'un bloc
STOP_EMPTY_RUN = 5         # stop table si on rencontre STOP_EMPTY_RUN lignes vides consécutives après le header
HEADER_SCAN_DEPTH = 6      # nb de premières lignes du bloc à tester pour choisir la meilleure ligne d'en-tête

# Filtrage de colonnes après extraction (pour supprimer col/col_2… vides)
MIN_NON_NULL_RATIO = 0.05  # garder une colonne si >= 5% de valeurs non nulles
MIN_NON_NULL_ABS   = 2     # ... ou au moins 2 valeurs non nulles (filet de sécurité)
# ============================================ #

def strip_accents_lower(s: str) -> str:
    if s is None or pd.isna(s):
        return ""
    s = str(s)
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))
    return s.lower().strip()

def safe_read_excel_all_sheets(path: Path):
    xls = pd.ExcelFile(path, engine="openpyxl")
    out = []
    for sheet in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet, header=None)
        out.append((sheet, df))
    return out

def pick_encoding(path: Path):
    try:
        from charset_normalizer import from_path
        res = from_path(str(path)).best()
        return res.encoding if res else None
    except Exception:
        return None

def read_csv_robust(path: Path):
    enc_guess = pick_encoding(path)
    candidates = [e for e in [enc_guess, "utf-8-sig", "utf-8", "cp1252", "latin-1", "iso-8859-1"] if e]
    for enc in candidates:
        try:
            return pd.read_csv(path, sep=None, engine="python", encoding=enc, header=None)
        except Exception:
            pass
    # dernier recours permissif
    return pd.read_csv(path, sep=None, engine="python", encoding="latin-1", encoding_errors="replace", header=None)

def is_tabular_row(row, min_cols=MIN_COLS):
    return row.notna().sum() >= min_cols

def choose_header_row(block: pd.DataFrame):
    """
    Choisit la "meilleure" ligne d’en-tête dans les HEADER_SCAN_DEPTH premières lignes du bloc :
      - favorise la ligne avec le plus de cellules non vides
      - favorise la diversité textuelle (moins de chiffres purs)
      - évite les 'Unnamed' / vides
    Retourne l'index relatif (dans block) de la ligne header choisie.
    """
    best_idx = None
    best_score = -1
    limit = min(len(block), HEADER_SCAN_DEPTH)
    for i in range(limit):
        row = block.iloc[i]
        vals = row.tolist()
        non_empty = sum(pd.notna(v) and str(v).strip() != "" for v in vals)
        texty = 0
        for v in vals:
            s = str(v).strip() if pd.notna(v) else ""
            if s == "" or s.lower().startswith("unnamed"):
                continue
            if re.search(r"[A-Za-zÀ-ÿ]", s):
                texty += 1
        score = non_empty * 2 + texty
        if score > best_score:
            best_score = score
            best_idx = i
    return best_idx if best_idx is not None else 0

def clean_columns(vals):
    """
    Nettoie / normalise les noms de colonnes, évite doublons.
    """
    cols = []
    seen = {}
    for v in vals:
        s = strip_accents_lower(v)
        s = s.replace("\n", " ")
        s = re.sub(r"\s+", " ", s).strip(" -_")
        if s == "" or s.startswith("unnamed"):
            s = "col"
        s = re.sub(r"[^a-z0-9_ ]", "", s)
        s = re.sub(r"\s+", "_", s).strip("_")
        if s == "":
            s = "col"
        if s in seen:
            seen[s] += 1
            s = f"{s}_{seen[s]}"
        else:
            seen[s] = 1
        cols.append(s)
    return cols

def detect_blocks(df: pd.DataFrame):
    """
    Détecte des blocs tabulaires génériques :
      - une séquence de >= MIN_CONSEC_ROWS lignes "tabulaires"
      - tolère des petits trous (ROW_EMPTY_TOL)
    Retourne une liste de (start_idx, end_idx) sur l'index du df.
    """
    if list(df.columns) != list(range(df.shape[1])):
        df = df.copy()
        df.columns = list(range(df.shape[1]))

    blocks = []
    consec = 0
    empties_inside = 0
    start = None

    for i in df.index:
        row = df.loc[i]
        if is_tabular_row(row):
            if start is None:
                start = i
                consec = 0
                empties_inside = 0
            consec += 1
        else:
            if start is not None:
                empties_inside += 1
                if empties_inside > ROW_EMPTY_TOL:
                    end = i - (empties_inside)
                    if end >= start and consec >= MIN_CONSEC_ROWS:
                        blocks.append((start, end))
                    start = None
                    consec = 0
                    empties_inside = 0

    if start is not None and consec >= MIN_CONSEC_ROWS:
        blocks.append((start, df.index[-1]))

    return blocks

def prune_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Supprime colonnes 100% vides et colonnes trop peu remplies (ratio faible)."""
    if df.empty:
        return df
    df2 = df.dropna(axis=1, how="all").copy()
    if df2.empty:
        return df2
    n = len(df2)
    keep = []
    for c in df2.columns:
        nnz = df2[c].notna().sum()
        if nnz >= max(MIN_NON_NULL_ABS, int(n * MIN_NON_NULL_RATIO)):
            keep.append(c)
    return df2[keep].copy() if keep else pd.DataFrame()

def carve_table_from_block(df_block: pd.DataFrame):
    """
    À partir d'un bloc "dense", choisit un header, applique les colonnes,
    puis s'arrête quand on rencontre STOP_EMPTY_RUN lignes vides consécutives.
    Nettoie ensuite les colonnes vides/quasi vides.
    """
    if df_block.empty:
        return None

    h_rel = choose_header_row(df_block)
    header_vals = df_block.iloc[h_rel].tolist()
    cols = clean_columns(header_vals)

    data = df_block.iloc[h_rel+1:].copy()
    data.columns = cols

    # stop à N lignes vides consécutives après le header
    empty_run = 0
    cut_idx = data.index[-1]
    for idx in data.index:
        if data.loc[idx].isna().all():
            empty_run += 1
            if empty_run >= STOP_EMPTY_RUN:
                cut_idx = idx - STOP_EMPTY_RUN
                break
        else:
            empty_run = 0

    data = data.loc[:cut_idx]

    # filtrer lignes trop vides (garde les lignes avec un minimum d'info)
    data = data[data.notna().sum(axis=1) >= MIN_COLS]

    # >>> prune colonnes vides/quasi vides (supprime col, col_2... 100% NaN)
    data = prune_columns(data)

    if data.empty:
        return None

    return data.reset_index(drop=True)

def find_tables_in_sheet(df_raw: pd.DataFrame):
    """
    Détecte toutes les tables génériques dans une feuille.
    """
    if list(df_raw.columns) != list(range(df_raw.shape[1])):
        df_raw = df_raw.copy()
        df_raw.columns = list(range(df_raw.shape[1]))

    blocks = detect_blocks(df_raw)
    tables = []
    for (start, end) in blocks:
        block = df_raw.loc[start:end, :]
        table = carve_table_from_block(block)
        if table is not None and table.shape[1] >= MIN_COLS and table.shape[0] >= 1:
            tables.append(table)
    return tables

def process_file(path: Path):
    results = []  # [(sheet, idx, df_table)]
    if path.suffix.lower() in (".xlsx", ".xlsm"):
        sheets = safe_read_excel_all_sheets(path)
        for sheet, df_raw in sheets:
            tables = find_tables_in_sheet(df_raw)
            for i, t in enumerate(tables, start=1):
                results.append((sheet, i, t))
    elif path.suffix.lower() == ".csv":
        df_raw = read_csv_robust(path)
        tables = find_tables_in_sheet(df_raw)
        for i, t in enumerate(tables, start=1):
            results.append((None, i, t))
    else:
        raise ValueError(f"Extension non gérée: {path.suffix}")
    return results

def main():
    if not ROOT_DIR.exists():
        print(f"[error] Dossier introuvable : {ROOT_DIR.resolve()}")
        return

    files = []
    for pat in ("*.xlsx", "*.xlsm", "*.csv"):
        files += [p for p in ROOT_DIR.rglob(pat) if p.is_file() and not p.name.startswith("~$")]

    print("=== data header: folder info ===")
    print(f"path: {ROOT_DIR.resolve()}")
    print(f"files_found: {len(files)}")

    if not files:
        print("[warn] Aucun fichier .xlsx/.xlsm/.csv trouvé.")
        return

    if EXPORT_TABLES:
        EXPORT_DIR.mkdir(parents=True, exist_ok=True)

    for f in sorted(files):
        print("\n=== file ===")
        print(f"{f.name}  ({f.resolve()})")

        try:
            tables = process_file(f)
        except Exception as e:
            print(f"[error] Lecture échouée pour {f.name}: {e}")
            continue

        if not tables:
            print("[info] Aucune table détectée")
            continue

        for sheet, idx, df in tables:
            title = f"{f.stem}__{sheet or 'sheet'}__table_{idx}"
            print(f"\n--- table detected ---")
            print(f"name: {title}")
            print(f"rows: {len(df)} | cols: {df.shape[1]}")
            print("columns:", ", ".join(map(str, df.columns.tolist())))

            if SHOW_SAMPLES:
                with pd.option_context("display.max_columns", 60, "display.width", 160):
                    print(df.head(8))

            if EXPORT_TABLES:
                out_path = EXPORT_DIR / f"{title}.csv"
                df.to_csv(out_path, index=False)
                print(f"[saved] {out_path}")

    print("\n=== done ===")

if __name__ == "__main__":
    main()


=== data header: folder info ===
path: C:\globasoft\aerotech\fic
files_found: 1

=== file ===
fichier qualité des trigrammes.xlsx  (C:\globasoft\aerotech\fic\fichier qualité des trigrammes.xlsx)

--- table detected ---
name: fichier qualité des trigrammes__PDG__table_1
rows: 20 | cols: 4
columns: edition, date, motif, redacteur
  edition                 date                                              motif    redacteur
0       1  2024-06-10 00:00:00                                           Création  S. BELMONTE
1       2  2024-09-16 00:00:00                           Ajout nouveaux arrivants    Y. RAGEOT
2       3  2024-10-21 00:00:00                                    Ajout couturier    Y. RAGEOT
3       4  2024-10-28 00:00:00  Mise à jour des dates d'entrée \nAjout des hab...    Y. RAGEOT
4       5  2024-12-16 00:00:00  Ajout nouvelle arrivante - 1 personne - Feriel...  F.BOULHABEL
5       6  2024-12-18 00:00:00                              Départ de Yann RAGEOT  S. BELMONTE
6    